<div class="alert alert-block alert-info">

# Imports

</div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import talos as ta
from talos.utils import hidden_layers, early_stopper

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, RMSprop
from keras.activations import relu, elu, linear
from keras.losses import binary_crossentropy

Using TensorFlow backend.


<div class="alert alert-block alert-info">

# Dataframes

</div>

In [2]:
df_features = pd.read_csv('./dataset/ted_main_features.csv',sep="|",quotechar='"')

In [3]:
df_features.columns

Index(['comments', 'duration', 'languages', 'num_speaker', 'views',
       'film_month', 'film_dayofweek', 'published_month',
       'published_dayofweek', 'event_TED',
       ...
       'world', 'worldwide', 'worth', 'write', 'writer', 'wrong', 'ye', 'year',
       'york', 'young'],
      dtype='object', length=580)

In [4]:
df_features.shape

(2550, 580)

In [5]:
df_features.head()

,comments,duration,languages,num_speaker,views,film_month,film_dayofweek,published_month,published_dayofweek,event_TED,...,world,worldwide,worth,write,writer,wrong,ye,year,york,young
0,10,810,1,1,418368,8,5,9,1,1,...,0,0,0,0,0,0,0,0,0,0
1,3,795,1,1,542088,8,5,9,4,1,...,0,0,0,0,0,0,0,0,0,0
2,8,723,4,1,707788,8,5,9,4,1,...,0,0,0,0,0,0,0,0,1,0
3,10,934,2,1,527314,8,5,9,2,1,...,0,0,0,0,0,0,0,0,0,0
4,33,722,4,1,613915,7,2,9,3,1,...,1,0,0,0,0,0,0,0,0,0


<div class="alert alert-block alert-info">

# Dummies

</div>

## Create Dummies

In [6]:
df_film_month = pd.get_dummies(df_features['film_month'],prefix='film_month',drop_first=True)
df_published_month = pd.get_dummies(df_features['published_month'],prefix='published_month',drop_first=True)

df_film_day = pd.get_dummies(df_features['film_dayofweek'],prefix='film_dayofweek',drop_first=True)
df_published_day = pd.get_dummies(df_features['published_dayofweek'],prefix='published_dayofweek',drop_first=True)

In [7]:
df_film_month.head(2)

,film_month_2,film_month_3,film_month_4,film_month_5,film_month_6,film_month_7,film_month_8,film_month_9,film_month_10,film_month_11,film_month_12
0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0


In [8]:
df_film_day.head(2)

,film_dayofweek_1,film_dayofweek_2,film_dayofweek_3,film_dayofweek_4,film_dayofweek_5,film_dayofweek_6
0,0,0,0,0,1,0
1,0,0,0,0,1,0


In [9]:
print(df_film_month.shape)
print(df_film_day.shape)

(2550, 11)
(2550, 6)


In [10]:
df_published_month.head(2)

,published_month_2,published_month_3,published_month_4,published_month_5,published_month_6,published_month_7,published_month_8,published_month_9,published_month_10,published_month_11,published_month_12
0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0


In [11]:
df_published_day.head(2)

,published_dayofweek_1,published_dayofweek_2,published_dayofweek_3,published_dayofweek_4,published_dayofweek_5,published_dayofweek_6
0,1,0,0,0,0,0
1,0,0,0,1,0,0


In [12]:
print(df_published_month.shape)
print(df_published_day.shape)

(2550, 11)
(2550, 6)


## Drop Columns

In [13]:
df_features = df_features.drop(columns=['film_month','film_dayofweek','published_month','published_dayofweek'])
df_features.head()

,comments,duration,languages,num_speaker,views,event_TED,event_TEDx,event_noTED,previous_talks,previous_talk_views,...,world,worldwide,worth,write,writer,wrong,ye,year,york,young
0,10,810,1,1,418368,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,795,1,1,542088,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,723,4,1,707788,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,10,934,2,1,527314,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33,722,4,1,613915,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## Join Dummies

In [14]:
df_features = df_features.merge(df_film_month, left_index=True, right_index=True)
df_features = df_features.merge(df_published_month, left_index=True, right_index=True)
df_features = df_features.merge(df_film_day, left_index=True, right_index=True)
df_features = df_features.merge(df_published_day, left_index=True, right_index=True)
df_features.head()

,comments,duration,languages,num_speaker,views,event_TED,event_TEDx,event_noTED,previous_talks,previous_talk_views,...,film_dayofweek_3,film_dayofweek_4,film_dayofweek_5,film_dayofweek_6,published_dayofweek_1,published_dayofweek_2,published_dayofweek_3,published_dayofweek_4,published_dayofweek_5,published_dayofweek_6
0,10,810,1,1,418368,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1,3,795,1,1,542088,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,8,723,4,1,707788,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,10,934,2,1,527314,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,33,722,4,1,613915,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


<div class="alert alert-block alert-info">

# Train, Test, Split

</div>

In [15]:
# Generamos la matriz X y el vector y
X = df_features.drop(columns=['views'])
y = df_features['views']

In [16]:
# Separamos train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [17]:
# Estandarizamos las variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Chequeamos las shapes
print('X train shape: ', X_train.shape)
print('X test shape: ', X_test.shape)
print('y train shape: ', y_train.shape)
print('y test shape: ', y_test.shape)

X train shape:  (1785, 609)
X test shape:  (765, 609)
y train shape:  (1785,)
y test shape:  (765,)


In [18]:
# Primero, definimos una función que permita construir el modelo, donde todos los valores de los hiperparámetros se obtienen del diccionario params 

def views_ted_talks_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
    
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation=params['activation']))
    
    hidden_layers(model, params, 1) # Necesario para explorar distintas configuraciones de capas ocultas
    
    model.add(Dense(1, activation=params['last_activation']))
    
    model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=['mae'])
    
    history = model.fit(x_train, y_train,
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=1,
                        callbacks=[early_stopper(params['epochs'])])

    return history, model

In [19]:
# Segundo, almacenamos en un diccionario el espacio de hiperparámetros a explorar

p = {'batch_size': [128, 16],
     'epochs': [3000, 5000],
     'activation':[relu, elu, linear],
     'last_activation': [linear]}

In [20]:
# Tercero, realizamos la búsqueda con la clase Scan provista por Talos

t = ta.Scan(x=X_train, y=y_train, model=views_ted_talks_model, params=p, experiment_name='views_ted_talks_model', seed=42)

  0%|                                                                                          | 0/324 [00:00<?, ?it/s]

Train on 1249 samples, validate on 536 samples
Epoch 1/500
1249/1249 [==============================] - ETA: 12s - loss: nan - mean_absolute_error: n - 2s 1ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 46us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 43us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 45us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 45us/step - loss: nan - mean_a

Epoch 41/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 39us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 42/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 39us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 43/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 38us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 44/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 45us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 45/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 44us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_

  0%|▎                                                                                 | 1/324 [00:05<27:11,  5.05s/it]

Train on 1249 samples, validate on 536 samples
Epoch 1/500
1249/1249 [==============================] - ETA: 3s - loss: nan - mean_absolute_error: na - 0s 373us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 54us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 52us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 52us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 50us/step - loss: nan - mean

...

Train on 1249 samples, validate on 536 samples
Epoch 1/500
1249/1249 [==============================] - ETA: 2s - loss: nan - mean_absolute_error: na - 0s 301us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 39us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 40us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 38us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 38us/step - loss: nan - mean

Epoch 41/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 40us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 42/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 41us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 43/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 43us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 44/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 46us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 45/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 46us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_

  2%|█▎                                                                                | 5/324 [00:22<22:58,  4.32s/it]

Train on 1249 samples, validate on 536 samples
Epoch 1/500
1249/1249 [==============================] - ETA: 4s - loss: nan - mean_absolute_error: na - 1s 444us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 48us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/500
1249/1249 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 47us/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/500
 128/1249 [==>...........................] - ETA: 0s - loss: nan - mean_absolute_error: nan

KeyboardInterrupt: 

In [ ]:
# Pandas DataFrame que resume la información correspondiente a cada prueba
t.data.head()